# <font color="red">**4. PCA FS - High-freq Forecast**</font>

**Author:** Osmar Bolivar

In [2]:
#%pip install kaleido

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go
import plotly.express as px

In [4]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


## **1. Monthly data**

In [5]:
raw = pd.read_csv("./Data/PC_DATASET.csv", index_col=0)
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 2010-01-01 to 2024-10-31
Data columns (total 87 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ipc_all  178 non-null    float64
 1   lag_1    5901 non-null   float64
 2   lag_2    5901 non-null   float64
 3   lag_3    5901 non-null   float64
 4   lag_6    5901 non-null   float64
 5   lag_9    5901 non-null   float64
 6   lag_12   5901 non-null   float64
 7   freq     5901 non-null   object 
 8   PC_1     5901 non-null   float64
 9   PC_2     5901 non-null   float64
 10  PC_3     5901 non-null   float64
 11  PC_4     5901 non-null   float64
 12  PC_5     5901 non-null   float64
 13  PC_6     5901 non-null   float64
 14  PC_7     5901 non-null   float64
 15  PC_8     5901 non-null   float64
 16  PC_9     5901 non-null   float64
 17  PC_10    5901 non-null   float64
 18  PC_11    5901 non-null   float64
 19  PC_12    5901 non-null   float64
 20  PC_13    5901 non-null   float64
 21  PC_1

In [6]:
df = raw.copy().query('freq == "month"')
df.drop(columns=["freq"], inplace=True)

df_w = raw.copy().query('freq == "week"')
df_w.drop(columns=["freq"], inplace=True)

df_d = raw.copy().query('freq == "day"')
df_d.drop(columns=["freq"], inplace=True)

In [7]:
isna = df.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [8]:
df.shape

(178, 86)

## **2. Train and Val sets**

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
#train_size = int(len(df) * 0.8)
#train, validation = df[:train_size], df[train_size:]
train, validation = train_test_split(df, test_size=0.2, random_state=42)

print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 142; variables in train set: 86
Obs in validation set: 36; variables in validation set: 86


In [11]:
test_w = df_w.copy()
test_d = df_d.copy()
print(f'Obs in weekly test set: {test_w.shape[0]}; variables in weekly test set: {test_w.shape[1]}')
print(f'Obs in daily test set: {test_d.shape[0]}; variables in daily test set: {test_d.shape[1]}')

Obs in weekly test set: 305; variables in weekly test set: 86
Obs in daily test set: 5418; variables in daily test set: 86


In [12]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
test_scaled_w = pd.DataFrame(scaler.transform(test_w), columns=test_w.columns, index=test_w.index)
test_scaled_d = pd.DataFrame(scaler.transform(test_d), columns=test_d.columns, index=test_d.index)

X_train = train_scaled.drop('ipc_all', axis=1)
y_train = train['ipc_all']
#y_train = train_scaled['ipc_all']

X_validation = validation_scaled.drop('ipc_all', axis=1)
y_validation = validation['ipc_all']
#y_validation = validation_scaled['ipc_all']

X_test_w = test_scaled_w.drop('ipc_all', axis=1)
X_test_d = test_scaled_d.drop('ipc_all', axis=1)

## **3. Algorithms**

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**   

**Without tuning (Validation):**
MSE:  2.209; R2:  0.001; MAE:  1.256

In [14]:
#ridge = Ridge(random_state=0)
ridge = Ridge(alpha=6.5, fit_intercept=True, positive=False, random_state=0)  
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

ridge_test_pred_w = ridge.predict(X_test_w)
ridge_test_pred_d = ridge.predict(X_test_d)

Train MSE:  0.05228182009710733
Train R2:  0.9902152223135229
Train MAE:  0.1738242430632907
Validation MSE:  0.18002187768103833
Validation R2:  0.9803945409217761
Validation MAE:  0.3123792057766494


In [15]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,lag_1,0.705719
1,lag_2,0.382657
2,lag_6,0.339039
3,lag_3,0.281201
4,PC_4,0.235389
5,PC_3,0.199229
6,PC_7,0.150298
7,PC_13,0.147967
8,PC_66,0.132821
9,PC_5,0.131093


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.796
Validation R2:  0.640
Validation MAE:  0.690

In [16]:
#lasso = Lasso(random_state=0)
lasso = Lasso(alpha=0.1, fit_intercept=True, max_iter=10000, random_state=0)  
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

lasso_test_pred_w = lasso.predict(X_test_w)
lasso_test_pred_d = lasso.predict(X_test_d)

Train MSE:  0.30604845777973616
Train R2:  0.9427216551546643
Train MAE:  0.40477550815095087
Validation MSE:  0.35839505745490063
Validation R2:  0.9609686349054792
Validation MAE:  0.4543185460207854


In [17]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,lag_1,2.024470
1,PC_5,0.036123
2,PC_66,0.035656
3,PC_20,0.004955
4,PC_49,-0.000000
5,PC_55,0.000000
6,PC_54,0.000000
7,PC_53,-0.000000
8,PC_52,0.000000
9,PC_51,-0.000000


### **3.4. ADA**
Without tuning:
Validation MSE:  0.324
Validation R2:  0.853
Validation MAE:  0.429

In [18]:
#ada = AdaBoostRegressor(random_state=0)
ada = AdaBoostRegressor(learning_rate=1.1, n_estimators=55, random_state=0)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

ada_test_pred_w = ada.predict(X_test_w)
ada_test_pred_d = ada.predict(X_test_d)

Train MSE:  0.11457484099370506
Train R2:  0.9785568033877822
Train MAE:  0.27877301916863056
Validation MSE:  0.38645377943067843
Validation R2:  0.9579128722805719
Validation MAE:  0.4896965021118274


In [19]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
feature_importance_ada.head(15)

,feat,imp_ada
0,lag_1,0.799128
1,PC_2,0.122421
2,PC_3,0.010449
3,PC_11,0.004163
4,PC_67,0.003981
5,PC_66,0.003705
6,PC_41,0.003452
7,PC_74,0.003260
8,PC_78,0.003134
9,lag_2,0.002772


### **3.5. GBR**
Without tuning:
Validation MSE:  0.277
Validation R2:  0.874
Validation MAE:  0.414

In [20]:
#gbr = GradientBoostingRegressor(random_state=0)
gbr = GradientBoostingRegressor(n_estimators=130, learning_rate=0.35, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

gbr_test_pred_w = gbr.predict(X_test_w)
gbr_test_pred_d = gbr.predict(X_test_d)

Train MSE:  6.238539182832533e-09
Train R2:  0.9999999988324293
Train MAE:  6.463712024062987e-05
Validation MSE:  0.3091981144392413
Validation R2:  0.9663264762161681
Validation MAE:  0.4481054396628972


In [21]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
feature_importance_gbr.head(15)

,feat_gbr,imp_gbr
0,lag_1,0.840474
1,PC_2,0.090901
2,lag_12,0.009679
3,PC_3,0.008478
4,PC_22,0.005767
5,PC_11,0.004931
6,PC_6,0.004343
7,PC_9,0.004114
8,PC_66,0.003954
9,PC_62,0.003669


### **3.4. RF**
Without tuning:
Validation MSE:  0.324
Validation R2:  0.853
Validation MAE:  0.434

In [22]:
#rf = RandomForestRegressor(random_state=0)
rf = RandomForestRegressor(max_depth=4, n_estimators=105, random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

rf_test_pred_w = rf.predict(X_test_w)
rf_test_pred_d = rf.predict(X_test_d)

Train MSE:  0.1168075798332237
Train R2:  0.978138936275733
Train MAE:  0.2665171036164991
Validation MSE:  0.46623696561211275
Validation R2:  0.9492240061718545
Validation MAE:  0.5486471240503082


In [23]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
feature_importance_rf.head(30)

,feat,imp_rf
0,lag_1,0.882860
1,PC_3,0.015605
2,PC_2,0.015548
3,PC_11,0.013808
4,lag_2,0.011796
5,PC_5,0.004876
6,PC_6,0.003796
7,PC_66,0.002750
8,PC_20,0.002632
9,lag_3,0.002001


### **3.5. ET**   
Without tuning:
Validation MSE:  0.322
Validation R2:  0.854
Validation MAE:  0.423

In [24]:
et = ExtraTreesRegressor(random_state=0)
#et = ExtraTreesRegressor(random_state=0)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

et_test_pred_w = et.predict(X_test_w)
et_test_pred_d = et.predict(X_test_d)

Train MSE:  2.3946812197784293e-29
Train R2:  1.0
Train MAE:  3.683477623074133e-15
Validation MSE:  0.2687879602600837
Validation R2:  0.9707273836742488
Validation MAE:  0.40492615782107777


In [25]:
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
feature_importance_et.head(30)

,feat,imp_et
0,lag_1,0.495900
1,lag_2,0.186232
2,PC_2,0.094625
3,lag_3,0.049994
4,lag_6,0.027562
5,PC_11,0.012992
6,PC_3,0.008757
7,PC_5,0.008568
8,lag_12,0.008563
9,PC_8,0.005994


## **4. Report**

In [26]:
val_forecast = pd.DataFrame(
    {'cpi': y_validation,
     'ridge': ridge_val_pred,
     'lasso': lasso_val_pred,
     'ada': ada_val_pred,
     'gbr': gbr_val_pred,
     'rf': rf_val_pred,
     'et': et_val_pred
    }, index=validation.index
)
val_forecast['set'] = 'validation'

week_forecast = pd.DataFrame(
    {'cpi': np.nan,
     'ridge': ridge_test_pred_w,
     'lasso': lasso_test_pred_w,
     'ada': ada_test_pred_w,
     'gbr': gbr_test_pred_w,
     'rf': rf_test_pred_w,
     'et': et_test_pred_w
    }, index=test_w.index
)
week_forecast['set'] = 'week'

day_forecast = pd.DataFrame(
    {'cpi': np.nan,
     'ridge': ridge_test_pred_d,
     'lasso': lasso_test_pred_d,
     'ada': ada_test_pred_d,
     'gbr': gbr_test_pred_d,
     'rf': rf_test_pred_d,
     'et': et_test_pred_d
    }, index=test_d.index
)
day_forecast['set'] = 'day'

all_forecast = pd.concat([val_forecast, week_forecast, day_forecast], axis=0)
all_forecast = all_forecast.sort_index(ascending=True)
all_forecast.tail(40)

,cpi,ridge,lasso,ada,gbr,rf,et,set
2024-09-27,NaN,6.935903,5.400885,5.826399,6.539438,5.766934,6.438672,day
2024-09-28,NaN,6.936882,5.403687,5.924581,6.698810,5.796210,6.450494,day
2024-09-29,NaN,6.849597,5.387030,5.829957,6.541128,5.763192,6.410840,week
2024-09-29,NaN,6.937835,5.403721,5.924581,6.698810,5.796210,6.444367,day
2024-09-30,NaN,6.926544,5.406939,6.130926,6.611596,5.780184,6.436358,day
2024-10-01,NaN,7.457304,6.282136,6.958121,7.724704,7.003099,7.158535,day
2024-10-02,NaN,7.449015,6.283677,7.017318,7.717150,7.070711,7.208960,day
2024-10-03,NaN,7.506257,6.288092,6.958121,7.636359,7.118285,7.284369,day
2024-10-04,NaN,7.518640,6.289782,7.184978,7.632057,7.108660,7.344052,day
2024-10-05,NaN,7.490340,6.283237,7.049409,7.512193,7.100368,7.325325,day


In [27]:
metrics = pd.DataFrame(
    {'MSE': [mse_val_ridge, mse_val_lasso, mse_val_ada, mse_val_gbr, mse_val_rf, mse_val_et],
     'R2': [r2_val_ridge, r2_val_lasso, r2_val_ada, r2_val_gbr, r2_val_rf, r2_val_et],
     'MAE': [mae_val_ridge, mae_val_lasso, mae_val_ada, mae_val_gbr, mae_val_rf, mae_val_et]},
    index=['ridge', 'lasso', 'ada', 'gbr', 'rf', 'et']
)
metrics.sort_values(['MSE'], ascending=True, inplace=True)
metrics

,MSE,R2,MAE
ridge,0.180022,0.980395,0.312379
et,0.268788,0.970727,0.404926
gbr,0.309198,0.966326,0.448105
lasso,0.358395,0.960969,0.454319
ada,0.386454,0.957913,0.489697
rf,0.466237,0.949224,0.548647


In [28]:
all_forecast['min'] = all_forecast[['ridge', 'lasso', 'ada', 'gbr', 'rf', 'et']].min(axis=1)
all_forecast['max'] = all_forecast[['ridge', 'lasso', 'ada', 'gbr', 'rf', 'et']].max(axis=1)

inv1 = (1/mse_val_ridge)  / 1000
inv2 = (1/mse_val_lasso) / 1000
inv3 = (1/mse_val_ada) / 1000
inv4 = (1/mse_val_gbr) / 1000
inv5 = (1/mse_val_rf) / 1000
inv6 = (1/mse_val_et) / 1000

num_1 = (all_forecast['ridge'] * inv1) + (all_forecast['lasso'] * inv2) + (all_forecast['ada'] * inv3) + (all_forecast['gbr'] * inv4) + (all_forecast['rf'] * inv5) + (all_forecast['et'] * inv6)
den_1 = inv1+inv2+inv3+inv4+inv5+inv6
all_forecast['w_avg'] = num_1 / den_1

num_2 = (all_forecast['ridge'] * inv1) + (all_forecast['et'] * inv6) + (all_forecast['gbr'] * inv4)
den_2 = inv1+inv6+inv4
all_forecast['w_avg_best'] = num_2 / den_2

all_forecast

,cpi,ridge,lasso,ada,gbr,rf,et,set,min,max,w_avg,w_avg_best
2010-01-01,NaN,0.394005,0.799776,0.576420,0.307726,0.443201,0.351035,day,0.307726,0.799776,0.457444,0.358919
2010-01-02,NaN,0.373202,0.805333,0.576420,0.290162,0.435434,0.416883,day,0.290162,0.805333,0.461012,0.364724
2010-01-03,NaN,0.371630,0.805480,0.576420,0.290162,0.435434,0.416883,day,0.290162,0.805480,0.460596,0.364026
2010-01-04,NaN,0.358394,0.796721,0.576420,0.310306,0.461511,0.371536,day,0.310306,0.796721,0.453328,0.349870
2010-01-05,NaN,0.440415,0.799171,0.611340,0.376054,0.442249,0.295510,day,0.295510,0.799171,0.475357,0.380679
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27,NaN,8.223429,6.337157,7.299910,7.851887,7.191548,7.644893,week,6.337157,8.223429,7.563486,7.955310
2024-10-28,NaN,8.641808,6.357099,7.299910,7.711070,7.129770,7.580376,day,6.357099,8.641808,7.640948,8.085499
2024-10-29,NaN,8.374321,6.345814,7.299910,7.581451,7.127379,7.600988,day,6.345814,8.374321,7.547847,7.939338
2024-10-30,NaN,8.497510,6.348985,7.049409,7.660199,7.174748,7.479467,day,6.348985,8.497510,7.545308,7.978259


In [29]:
all_forecast['cpi'] = df['ipc_all'].reindex(all_forecast.index)
all_forecast.tail(35)

,cpi,ridge,lasso,ada,gbr,rf,et,set,min,max,w_avg,w_avg_best
2024-10-01,NaN,7.457304,6.282136,6.958121,7.724704,7.003099,7.158535,day,6.282136,7.724704,7.168189,7.437581
2024-10-02,NaN,7.449015,6.283677,7.017318,7.717150,7.070711,7.208960,day,6.283677,7.717150,7.189137,7.446944
2024-10-03,NaN,7.506257,6.288092,6.958121,7.636359,7.118285,7.284369,day,6.288092,7.636359,7.204033,7.473902
2024-10-04,NaN,7.518640,6.289782,7.184978,7.632057,7.108660,7.344052,day,6.289782,7.632057,7.246365,7.496039
2024-10-05,NaN,7.490340,6.283237,7.049409,7.512193,7.100368,7.325325,day,6.283237,7.512193,7.196377,7.446913
2024-10-06,NaN,7.498616,6.283830,7.049409,7.512193,7.079696,7.359537,day,6.283830,7.512193,7.202895,7.460763
2024-10-06,NaN,7.504327,6.284886,7.017318,7.649796,7.145704,7.269220,week,6.284886,7.649796,7.212990,7.472014
2024-10-07,NaN,7.495766,6.285423,7.097941,7.638216,7.210731,7.411251,day,6.285423,7.638216,7.252576,7.507460
2024-10-08,NaN,7.351539,6.291451,7.184978,7.893472,7.150539,7.394010,day,6.291451,7.893472,7.256229,7.504281
2024-10-09,NaN,7.278836,6.287625,7.140826,7.873597,7.067595,7.313766,day,6.287625,7.873597,7.202839,7.442993


In [30]:
only_validation = all_forecast.query('set == "validation"')

mse_val_w_avg = mean_squared_error(only_validation['cpi'], only_validation['w_avg'])
r2_val_w_avg = r2_score(only_validation['cpi'], only_validation['w_avg'])
mae_val_w_avg = mean_absolute_error(only_validation['cpi'], only_validation['w_avg'])

mse_val_w_avg_best = mean_squared_error(only_validation['cpi'], only_validation['w_avg_best'])
r2_val_w_avg_best = r2_score(only_validation['cpi'], only_validation['w_avg_best'])
mae_val_w_avg_best = mean_absolute_error(only_validation['cpi'], only_validation['w_avg_best'])

metrics_b = pd.DataFrame(
    {'MSE': [mse_val_w_avg, mse_val_w_avg_best],
     'R2': [r2_val_w_avg, r2_val_w_avg_best],
     'MAE': [mae_val_w_avg, mae_val_w_avg_best]},
    index=['w_avg', 'w_avg_best']
)

metrics_all = pd.concat([metrics, metrics_b], axis=0)
metrics_all.sort_values(['MSE'], ascending=True, inplace=True)
metrics_all

,MSE,R2,MAE
w_avg_best,0.176635,0.980763,0.340050
ridge,0.180022,0.980395,0.312379
w_avg,0.220571,0.975979,0.379782
et,0.268788,0.970727,0.404926
gbr,0.309198,0.966326,0.448105
lasso,0.358395,0.960969,0.454319
ada,0.386454,0.957913,0.489697
rf,0.466237,0.949224,0.548647


In [31]:
# Create a scatter plot
fig = px.scatter(only_validation, x='cpi', y='w_avg_best')
# Fit a line to the data
fit = np.polyfit(only_validation['cpi'], only_validation['w_avg_best'], deg=1)
fit_fn = np.poly1d(fit)
# Add the fitted line to the plot
fig.add_trace(go.Scatter(
    x=only_validation['cpi'], 
    y=fit_fn(only_validation['cpi']), 
    mode='lines', 
    name='Fitted Line',
    line=dict(color='red', width=2)
))

# Update layout for better visualization
fig.update_layout(
    xaxis_title='Observed y-o-y Inflation',
    yaxis_title='Weighted Average Best Forecast',
    template='plotly_white',
    width=800, 
    height=600,
    legend=dict(
        x=0.8,
        y=0.1,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='black',
        borderwidth=1
    ),
    title=dict(
        x=0.5,
        xanchor='center'
    )
)

# Show the plot
fig.show()

In [32]:
weekly_forecast = all_forecast.copy().query('set == "week"')
daily_forecast = all_forecast.copy().query('set == "day"')

#### Weekly Forecast

In [33]:
fig1 = go.Figure()
# Add the forecast interval fill
fig1.add_trace(go.Scatter(
    x=weekly_forecast.index.tolist() + weekly_forecast.index[::-1].tolist(),
    y=weekly_forecast["min"].tolist() + weekly_forecast["max"][::-1].tolist(),
    fill='toself',
    fillcolor='#ffdabe',
    line=dict(color='rgba(255,255,255,0.5)'),
    hoverinfo="skip",
    showlegend=True,
    name="Forecast Interval"
))
# Add the high-frequency forecast line
fig1.add_trace(go.Scatter(
    x=weekly_forecast.index, 
    y=weekly_forecast["w_avg_best"], 
    mode='lines+markers', 
    name="Weekly Forecast", 
    line=dict(color='#ff7d00', width=1.0),
    marker=dict(size=2.5)
))
# Add the y-o-y inflation line
fig1.add_trace(go.Scatter(
    x=df.loc['2018-12-30':, ].index, 
    y=df.loc['2018-12-30':, "ipc_all"], 
    mode='markers', 
    name="Observed y-o-y Inflation", 
    line=dict(color='#073763', width=0.8),
    marker=dict(size=4)
))
# Update layout for better visualization
fig1.update_layout(
    #title='High-frequency Inflation Forecast and y-o-y Inflation',
    #xaxis_title='Date',
    #yaxis_title='%',
    #legend_title='Legend',
    template='plotly_white',
    width=1200, 
    height=600,
    legend=dict(
        x=0.6,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        font=dict(size=15),
        bordercolor='black',
        borderwidth=1
    )
)
# Update x-axis to show dates in the desired format
fig1.update_xaxes(tickformat='%b %Y')
# Show the plot
fig1.show()

# Save the plot as an image
fig1.write_image('./Images/F2_Weekly_Forecast.png', format='png', scale=3)

#### Daily Forecast

In [34]:
fig2 = go.Figure()
# Add the forecast interval fill
fig2.add_trace(go.Scatter(
    x=daily_forecast.index.tolist() + daily_forecast.index[::-1].tolist(),
    y=daily_forecast["min"].tolist() + daily_forecast["max"][::-1].tolist(),
    fill='toself',
    fillcolor='#ffdabe',
    line=dict(color='rgba(255,255,255,0.5)'),
    hoverinfo="skip",
    showlegend=True,
    name="Forecast Interval"
))
# Add the high-frequency forecast line
fig2.add_trace(go.Scatter(
    x=daily_forecast.index, 
    y=daily_forecast["w_avg_best"], 
    mode='lines', 
    name="Daily Forecast", 
    line=dict(color='#ff7d00', width=1.0),
    #marker=dict(size=1.5)
))
# Add the y-o-y inflation line
fig2.add_trace(go.Scatter(
    x=df.index, 
    y=df["ipc_all"], 
    mode='markers', 
    name="Observed y-o-y Inflation", 
    line=dict(color='#073763', width=0.8),
    marker=dict(size=4)
))
# Update layout for better visualization
fig2.update_layout(
    #title='High-frequency Inflation Forecast and y-o-y Inflation',
    #xaxis_title='Date',
    #yaxis_title='%',
    #legend_title='Legend',
    template='plotly_white',
    width=1200, 
    height=600,
    legend=dict(
        x=0.75,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        font=dict(size=15),
        bordercolor='black',
        borderwidth=1
    )
)
# Update x-axis to show dates in the desired format
fig2.update_xaxes(tickformat='%b %Y')
# Show the plot
fig2.show()

# Save the plot as an image
fig2.write_image('./Images/F2_Daily_Forecast.png', format='png', scale=3)

In [35]:
all_forecast.to_csv('./Forecast/F2_FORECAST.csv')

#### End